## Hands-on Modul 2: Eksplorasi Tokenization & Sampling
_Repository GitHub: https://github.com/Komdigi-x-BINAR-LLM/Module-02_

**Tujuan:** Memvisualisasikan konsep Tokenization dan efek Sampling pada LLM.

# Instalasi Library

In [1]:
# Setup Awal (Jalankan sel ini jika di Colab atau environment baru)
# !pip install transformers torch notebook

In [2]:
# Import library
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import logging
import sys
import importlib

# Memaksa Konfigurasi Ulang Logging
importlib.reload(logging)
logging.basicConfig(
    level=logging.INFO, # Tampilkan pesan INFO
    format='%(levelname)s: %(message)s',
    stream=sys.stdout # Paksa output ke stdout (yang ditangkap notebook)
)
# -----------------------------

logging.info("Setup selesai. Library siap digunakan.")

INFO: Setup selesai. Library siap digunakan.


---
### Bagian 1: Eksplorasi Tokenization (Konsep 2.2)
Kita akan lihat bagaimana tokenizer Hugging Face memecah teks menjadi token, termasuk subword.

# Memuat Tokenizer

In [3]:
# Pilih tokenizer (coba ganti-ganti!)
# tokenizer_name = "bert-base-uncased"
tokenizer_name = "Qwen/Qwen2.5-0.5B-Instruct" # Contoh lain
# tokenizer_name = "indobenchmark/indobert-base-p1" # Contoh tokenizer Bahasa Indonesia

try:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    logging.info(f"Tokenizer '{tokenizer_name}' berhasil dimuat.")
    # Cek & set pad_token jika perlu (penting nanti untuk generate)
    if tokenizer.pad_token is None:
        if tokenizer.eos_token:
            tokenizer.pad_token = tokenizer.eos_token
            logging.info("Menggunakan eos_token sebagai pad_token.")
        else:
            # Jika eos_token juga tidak ada, tambahkan token baru
            tokenizer.add_special_tokens({'pad_token': '[PAD]'})
            logging.warning("Menambahkan [PAD] sebagai pad_token baru.")

except Exception as e:
    logging.error(f"Gagal memuat tokenizer '{tokenizer_name}': {e}")
    # Berhenti di sini jika tokenizer gagal dimuat
    raise

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

INFO: Tokenizer 'Qwen/Qwen2.5-0.5B-Instruct' berhasil dimuat.


In [4]:
kalimat_contoh = [
    "Pembelajaran mesin berkembang pesat.",
    "Unbelievably complex tokenization!",
    "Ayo makan nasi goreng.",
    "Kantor Kominfo ada di Jakarta." # Contoh dengan nama/lokasi
]

for kalimat in kalimat_contoh:
    print(f"\nKalimat Input: '{kalimat}'")
    # Tokenisasi (mendapatkan dictionary)
    hasil_tokenisasi = tokenizer(kalimat)
    input_ids = hasil_tokenisasi['input_ids']
    print(f"-> input_ids: {input_ids}")

    # Konversi ID kembali ke token untuk dilihat
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # Beberapa tokenizer menambahkan spasi khusus (misal Ġ), kita bisa hilangkan utk kejelasan
    tokens_cleaned = [t.replace('Ġ', '') for t in tokens]
    print(f"-> Tokens     : {tokens_cleaned}") # Token setelah dibersihkan
    print(f"-> Jumlah Tokens: {len(tokens)}")


Kalimat Input: 'Pembelajaran mesin berkembang pesat.'
-> input_ids: [47, 8858, 301, 84033, 10846, 258, 96583, 8858, 524, 18050, 266, 13]
-> Tokens     : ['P', 'emb', 'el', 'ajaran', 'mes', 'in', 'berk', 'emb', 'ang', 'pes', 'at', '.']
-> Jumlah Tokens: 12

Kalimat Input: 'Unbelievably complex tokenization!'
-> input_ids: [1806, 31798, 88134, 6351, 3950, 2022, 0]
-> Tokens     : ['Un', 'belie', 'vably', 'complex', 'token', 'ization', '!']
-> Jumlah Tokens: 7

Kalimat Input: 'Ayo makan nasi goreng.'
-> input_ids: [32, 16032, 296, 19262, 308, 10215, 90432, 968, 13]
-> Tokens     : ['A', 'yo', 'm', 'akan', 'n', 'asi', 'gore', 'ng', '.']
-> Jumlah Tokens: 9

Kalimat Input: 'Kantor Kominfo ada di Jakarta.'
-> input_ids: [42, 88167, 26692, 2733, 34234, 1853, 63033, 13]
-> Tokens     : ['K', 'antor', 'Kom', 'info', 'ada', 'di', 'Jakarta', '.']
-> Jumlah Tokens: 8


**Observasi:**
* Perhatikan bagaimana kata-kata dipecah. Apakah ada yang menjadi *subword* (ditandai `##` di BERT, atau cara lain di model berbeda)?
* Kata umum vs. kata langka/nama, bagaimana tokenisasinya?
* Apakah ada token spesial yang ditambahkan (`[CLS]`, `[SEP]` untuk BERT, `<|endoftext|>` untuk GPT/Qwen)?
* **Eksperimen:** Coba masukkan kalimat Anda sendiri di sel atas dan lihat hasilnya!

In [16]:
# Pilih model generatif KECIL (sesuaikan jika environment kuat)
# model_name_gen = "distilgpt2" # Sangat ringan, tapi mungkin kurang pintar
model_name_gen = "Qwen/Qwen2.5-0.5B-Instruct" # Lebih baik jika RAM/VRAM cukup

try:
    # Muat tokenizer & model untuk generasi
    tokenizer_gen = AutoTokenizer.from_pretrained(model_name_gen)
    model_gen = AutoModelForCausalLM.from_pretrained(model_name_gen)
    logging.info(f"Model generatif '{model_name_gen}' berhasil dimuat.")

    # Set pad_token jika belum ada
    if tokenizer_gen.pad_token is None:
        if tokenizer_gen.eos_token:
            tokenizer_gen.pad_token = tokenizer_gen.eos_token
            logging.info("Menggunakan eos_token sebagai pad_token untuk generasi.")
        else:
            tokenizer_gen.add_special_tokens({'pad_token': '[PAD]'})
            model_gen.resize_token_embeddings(len(tokenizer_gen)) # Penting jika tambah token
            logging.warning("Menambahkan [PAD] dan resize embedding model.")

    # Siapkan device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_gen.to(device)
    logging.info(f"Model generatif dipindahkan ke {device}.")

    prompt = "Kucing adalah jenis"
    logging.info(f"\nPrompt Awal: '{prompt}'")

    model_ready = True # Tandai model siap

except Exception as e:
    logging.error(f"Gagal memuat atau menjalankan model generatif: {e}", exc_info=True)
    print("\n[INFO] Bagian Eksplorasi Sampling tidak dapat dijalankan.")
    print("      Pastikan Anda memiliki koneksi internet yang stabil dan RAM/VRAM yang cukup.")
    print("      Jika menggunakan Colab gratis, coba restart Runtime dan pilih GPU.")
    model_ready = False # Tandai model tidak siap

INFO: loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-0.5B-Instruct/snapshots/7ae557604adf67be50417f59c2c2f167def9a775/model.safetensors
INFO: Model generatif 'Qwen/Qwen2.5-0.5B-Instruct' berhasil dimuat.
INFO: Model generatif dipindahkan ke cpu.
INFO: 
Prompt Awal: 'Kucing adalah jenis'


---
### Bagian 2: Eksplorasi Sampling (Konsep 2.4)
Kita akan gunakan model generatif kecil untuk melihat efek parameter sampling.
**Peringatan:** Menjalankan model butuh RAM/VRAM yang cukup. Jika Colab gratis, mungkin perlu *runtime* GPU.

# Greedy Search

In [17]:
if model_ready:
    with torch.no_grad(): # Selalu pakai no_grad() saat inference
        # 1. Greedy Search (do_sample=False)
        print("\n--- 1. Greedy Search (Deterministik) ---")
        for i in range(2): # Jalankan 2 kali
            output_greedy = model_gen.generate(
                **tokenizer_gen(prompt, return_tensors="pt").to(device),
                max_new_tokens=30,
                pad_token_id=tokenizer_gen.pad_token_id,
                do_sample=False
            )
            print(f"Run {i+1}: {tokenizer_gen.decode(output_greedy[0], skip_special_tokens=True)}")
else:
    print("Model tidak siap, lewati Greedy Search.")


--- 1. Greedy Search (Deterministik) ---
Run 1: Kucing adalah jenis kucing yang sangat populer di Indonesia. Kucing ini memiliki banyak kelebihan dan kekurangan, baik dari segi kesehat
Run 2: Kucing adalah jenis kucing yang sangat populer di Indonesia. Kucing ini memiliki banyak kelebihan dan kekurangan, baik dari segi kesehat


# Temperature Tinggi

In [18]:
if model_ready:
    with torch.no_grad():
        # 2. Sampling Temperature Tinggi (Lebih Acak)
        print("\n--- 2. Sampling (Temp=1.5, TopK=0) ---")
        for i in range(2):
            output_high_temp = model_gen.generate(
                **tokenizer_gen(prompt, return_tensors="pt").to(device),
                max_new_tokens=20,
                pad_token_id=tokenizer_gen.pad_token_id,
                do_sample=True,
                temperature=1.5, # Sangat tinggi, bisa mendongeng
                top_k=0 # Top-k 0 berarti tidak aktif
            )
            print(f"Run {i+1}: {tokenizer_gen.decode(output_high_temp[0], skip_special_tokens=True)}")
else:
    print("Model tidak siap, lewati Sampling Temp Tinggi.")


--- 2. Sampling (Temp=1.5, TopK=0) ---
Run 1: Kucing adalah jenis饲养实践中有很大程度优势和随机产生的种群,由爱尔兰荷尔德曼犬寡分支
Run 2: Kucing adalah jenis sayap parem biaya utama cost kingkiller %()->configVars','%******/
++warehouse<<


# Temperature Rendah

In [19]:
if model_ready:
    with torch.no_grad():
        # 3. Sampling Temperature Rendah (Lebih Fokus)
        print("\n--- 3. Sampling (Temp=0.5, TopK=0) ---")
        for i in range(2):
            output_low_temp = model_gen.generate(
                **tokenizer_gen(prompt, return_tensors="pt").to(device),
                max_new_tokens=20,
                pad_token_id=tokenizer_gen.pad_token_id,
                do_sample=True,
                temperature=0.5, # Lebih rendah, lebih fokus
                top_k=0
            )
            print(f"Run {i+1}: {tokenizer_gen.decode(output_low_temp[0], skip_special_tokens=True)}")
else:
    print("Model tidak siap, lewati Sampling Temp Rendah.")


--- 3. Sampling (Temp=0.5, TopK=0) ---
Run 1: Kucing adalah jenis kecil dan bergerak dengan cepat. Namun, mereka juga memiliki beberapa keterampil
Run 2: Kucing adalah jenis kecil yang memiliki kemampuan untuk bermain dengan baik. Mereka dapat melihat dan


# Top-k

In [20]:
if model_ready:
    with torch.no_grad():
        # 4. Sampling Top-k (Filter Kandidat)
        print("\n--- 4. Sampling (Temp=1.0, TopK=50) ---")
        output_topk = model_gen.generate(
            **tokenizer_gen(prompt, return_tensors="pt").to(device),
            max_new_tokens=20,
            pad_token_id=tokenizer_gen.pad_token_id,
            do_sample=True,
            temperature=1.0, # Default temp
            top_k=50 # Hanya pilih dari 50 kata paling mungkin
        )
        print(tokenizer_gen.decode(output_topk[0], skip_special_tokens=True))
else:
    print("Model tidak siap, lewati Sampling Top-k.")


--- 4. Sampling (Temp=1.0, TopK=50) ---
Kucing adalah jenis kecil yang seringkali diingatkan sebagai bahan utama dalam sistem diet s


# Top-p

In [21]:
if model_ready:
    with torch.no_grad():
        # 5. Sampling Top-p (Nucleus) - Paling Umum
        print("\n--- 5. Sampling (Temp=1.0, TopP=0.9) ---")
        output_topp = model_gen.generate(
            **tokenizer_gen(prompt, return_tensors="pt").to(device),
            max_new_tokens=20,
            pad_token_id=tokenizer_gen.pad_token_id,
            do_sample=True,
            temperature=1.0, # Default temp
            top_p=0.9, # Pilih dari kata teratas hingga total prob 90%
            top_k=0 # Matikan top-k jika pakai top-p
        )
        print(tokenizer_gen.decode(output_topp[0], skip_special_tokens=True))
else:
    print("Model tidak siap, lewati Sampling Top-p.")


--- 5. Sampling (Temp=1.0, TopP=0.9) ---
Kucing adalah jenis burung yang berbaya dengan keunikan khas yang sama - ikat tangan tan


**Observasi:**
* Bandingkan hasil *Greedy Search* (selalu sama) dengan metode *sampling* lainnya (bervariasi).
* Lihat bagaimana *Temperature* tinggi bisa menghasilkan teks yang lebih aneh/tidak koheren.
* Perhatikan bagaimana *Top-k* dan *Top-p* cenderung menghasilkan teks yang lebih masuk akal dibandingkan *Temperature* tinggi saja.
* **Eksperimen:** Coba ubah nilai `temperature`, `top_k`, `top_p`, atau `max_new_tokens`. Coba juga *prompt* yang berbeda!